### Hotel in Jeju Data
- '부킹닷컴'의 제주 내 호텔 성급, 가격, 위치, 후기를 스크래핑합니다.

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import InvalidSessionIdException
from time import sleep
import re
import json
import pandas as pd

In [122]:
url = 'https://www.booking.com/searchresults.ko.html?label=gen173nr-1DCAEoggI46AdIM1gEaH2IAQGYARe4AQfIAQzYAQPoAQGIAgGoAgO4Ar3AsYMGwAIB0gIkNjQ0OTNkZTgtOWUzNC00ODBjLTlmZjktYjA1ZjcwOTFmN2Vk2AIE4AIB&sid=febadbcd7e51aee26acbfeeea6e8299e&sb=1&src=searchresults&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.ko.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaH2IAQGYARe4AQfIAQzYAQPoAQGIAgGoAgO4Ar3AsYMGwAIB0gIkNjQ0OTNkZTgtOWUzNC00ODBjLTlmZjktYjA1ZjcwOTFmN2Vk2AIE4AIB%3Bsid%3Dfebadbcd7e51aee26acbfeeea6e8299e%3Btmpl%3Dsearchresults%3Bclass_interval%3D1%3Bdest_id%3D4170%3Bdest_type%3Dregion%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dregion%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsearch_selected%3D1%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dindex%3Bsrc_elem%3Dsb%3Bsrpvid%3Dadca5de230db0091%3Bss%3D%25EC%25A0%259C%25EC%25A3%25BC%25EB%258F%2584%252C%2520%25EB%258C%2580%25ED%2595%259C%25EB%25AF%25BC%25EA%25B5%25AD%3Bss_all%3D0%3Bssb%3Dempty%3Bsshis%3D0%3Btop_ufis%3D1%26%3B&ss=%EC%A0%9C%EC%A3%BC%EB%8F%84&is_ski_area=0&ssne=%EC%A0%9C%EC%A3%BC%EB%8F%84&ssne_untouched=%EC%A0%9C%EC%A3%BC%EB%8F%84&dest_id=4170&dest_type=region&checkin_year=2021&checkin_month=4&checkin_monthday=24&checkout_year=2021&checkout_month=4&checkout_monthday=25&group_adults=2&group_children=0&no_rooms=1&from_sf=1'
path = '/Users/seyoung/2021/dss17/eda_etc/chromedriver'
# path_f = '/Users/seyoung/2021/dss17/eda_etc/geckodriver'
driver = webdriver.Chrome(path)
# driver = webdriver.Firefox(executable_path=path_f)
driver.get(url)

In [123]:
page_bar = driver.find_elements_by_css_selector('#search_results_table > div.bui-pagination.results-paging_simplified.js-results-paging > nav > ul > li.bui-pagination__pages > ul')[0]
page_bar

<selenium.webdriver.remote.webelement.WebElement (session="b6190c0d9f42bda3438213cb0ea95919", element="fefdc422-047d-4158-b146-51b1c871161f")>

In [124]:
# 전체 페이지 개수 확인
try:
    pages = page_bar.find_elements_by_css_selector('a > div.bui-u-inline')
    last_pages = ''

    for idx, page in enumerate(pages):
        print(page.text)
        if idx == len(pages)-1: # 마지막 페이지
            last_pages = page.text
except:
    pages = 0
    last_pages = 0    

1
2
3
4
5
18


In [125]:
# 현재 페이지 번호
def get_curr():
    try:
        curr_page = driver.find_elements_by_css_selector('#search_results_table > div.bui-pagination.results-paging_simplified.js-results-paging > nav > ul > li.bui-pagination__pages > ul > li.bui-pagination__item.bui-pagination__item--active.sr_pagination_item.current > a > div.bui-u-inline')[0].text
    except:
        curr_page = 0
    return int(curr_page)

In [126]:
# 성급 (성급이 표시되지 않은 호텔 제외)
def get_stars(listed):
    stars = []

    for i in range(1, len(listed)):
        try:
            like = driver.find_element_by_xpath('//*[@id="hotellist_inner"]/div['+ str(i) +']/div[2]/div[1]/div[1]/div[1]/span/span[1]/span/span')
            a = like.get_attribute('aria-label')
            stars.append(a[0])
            
        # 성급이 표시되지 않는 호텔 
        except Exception as NoSuchElementException:
            stars.append('')
    
    return stars

In [127]:
# 가격
def get_prices(listed):
    prices = []
    
    for i in range(1, len(listed)):
        try:
            price = listed[0].find_elements_by_xpath('//*[@id="hotellist_inner"]/div['+ str(i) +']/div[2]/div[2]/div/div/div/div/div[2]/div[1]/div[2]/div/div')[0]
            base_price = listed[0].find_elements_by_xpath('//*[@id="hotellist_inner"]/div['+ str(i) +']/div[2]/div[2]/div/div/div/div/div[2]/div[1]/div[2]/div/span')[0]
            
            # strong label로 표시된 가격 포함
            if base_price.text == '':
                base_price = listed[0].find_elements_by_xpath('//*[@id="hotellist_inner"]/div[' + str(i) + ']/div[2]/div[2]/div/table/tbody/tr/td[2]/div[2]/strong/label')[0]
            
            prices.append(base_price.text)
            
        # 중간 베너 제외
        except Exception as IndexError:
            prices.append('')
            
    return prices

In [128]:
# 평점
def get_grades(listed):
    grades = []
    
    for i in range(1, len(listed)):
        try:
            grade = listed[0].find_elements_by_xpath('//*[@id="hotellist_inner"]/div[' + str(i) + ']/div[2]/div[1]/div[2]/div/div/a/div/div[1]')[0]
            grades.append(grade.text)
            
        except Exception as IndexError:
            grades.append('')
            
    return grades

In [129]:
# 호텔명
def get_names(listed):
    names = []
    
    for i in range(1, len(listed)):
        try:
            name = listed[0].find_elements_by_xpath('//*[@id="hotellist_inner"]/div['+ str(i) +']/div[2]/div[1]/div[1]/div[1]/h3/a/span[1]')[0]
            names.append(name.text)
            
        # 중간 베너 제외
        except Exception as IndexError:
            names.append('')
            
    return names

In [130]:
# 이용객 후기
def get_reviews():
    good_reviews, bad_reviews, res = [], [], []
    good_review, bad_review = '', ''
    review = driver.find_elements_by_xpath('//*[@id="review_list_page_container"]/ul/li')
    
    for i in range(len(review)):
        
        try:
            good_review += driver.find_elements_by_xpath('//*[@id="review_list_page_container"]/ul/li['+ str(i) +']/div/div[2]/div[2]/div[2]/div/div[1]/p/span[3]')[0].text +'\n'
        
        except Exception as IndexError:
            good_review += ''
            
        try:
            bad_review += driver.find_elements_by_xpath('//*[@id="review_list_page_container"]/ul/li['+ str(i) +']/div/div[2]/div[2]/div[2]/div/div[2]/p/span[3]')[0].text +'\n'
            
        except Exception as IndexError:
            bad_review += ''
        
    good_reviews.append(good_review)
    bad_reviews.append(bad_review)
    
    res.append(good_reviews)
    res.append(bad_reviews)
    
    return res

In [131]:
# 주소
def get_addrs():
    
    addr = driver.find_elements_by_xpath('//*[@id="showMap2"]/span[1]')[0]

    return addr.text

In [132]:
# 새 탭 열기
def open_newTab(listed):
    
    addrs, res, res_reviews = [], [], []
    
    for i in range(1, len(listed)):

        try:
            href = driver.find_element_by_xpath('//*[@id="hotellist_inner"]/div['+ str(i) +']/div[2]/div[1]/div[1]/div[1]/h3/a')
            
        except Exception as NoSuchElementException:
                addrs.append('')
                res_reviews.append('')
                continue
        else:
            href.send_keys(Keys.CONTROL + '\n') 

           # 새로운 탭으로 이동 (2번째 탭)
            driver.switch_to.window(driver.window_handles[1])  

            res_addrs = get_addrs()
            addrs.append(res_addrs)

            # 이용 후기 메뉴로 이동
            comment_btn = driver.find_element_by_xpath('//*[@id="show_reviews_tab"]')
            driver.execute_script("arguments[0].click();", comment_btn)

            res_reviews = get_reviews()
            res.append(res_reviews)
    
            driver.implicitly_wait(1)
            time.sleep(1+random.uniform(0,1))

            driver.close()
            driver.switch_to_window(driver.window_handles[0])
        
    return list(zip(res, addrs))

In [133]:
# 모든 페이지 스크래핑
def data_scraping():
    res_stars, res_names, res_prices, res_grades, res_addrs, res_goodReviews, res_badReviews = [], [], [], [], [], [], []
    
    root_div = driver.find_elements_by_css_selector('#hotellist_inner')[0]
    # 한 페이지당 호출되는 목록 수
    listed = root_div.find_elements_by_xpath('//*[@id="hotellist_inner"]/div')
    
    res_stars = get_stars(listed)
    res_names = get_names(listed)
    res_prices = get_prices(listed)
    res_grades = get_grades(listed)
    
    res = open_newTab(listed)
    
    for i in range(len(res)):
        res_addrs.append(res[i][1])
        res_goodReviews.append(res[i][0][0])
        res_badReviews.append(res[i][0][1])
    
#     print("data_scraping():", res_stars, res_names)
    
    return list(zip(res_names, res_stars, res_prices, res_grades, res_addrs, res_goodReviews, res_badReviews))

# data_scraping()

In [134]:
def get_hotelDatas():
    raw_datas = []
    
#     # 스크래핑 페이지 범위 (현재 페이지 ~ 마지막 페이지)
#     while get_curr() <= int(last_pages):

    # scraping
    result = data_scraping()
    raw_datas.append(result)

#     if get_curr() == int(last_pages):
#         break

    try:
        curr_page = driver.find_elements_by_css_selector('#search_results_table > div.bui-pagination.results-paging_simplified.js-results-paging > nav > ul > li.bui-pagination__pages > ul > li.bui-pagination__item.bui-pagination__item--active.sr_pagination_item.current > a > div.bui-u-inline')[0]
        print(curr_page.text,"페이지 완료")

        move_next = driver.find_elements_by_css_selector('#search_results_table > div.bui-pagination.results-paging_simplified.js-results-paging > nav > ul > li.bui-pagination__item.bui-pagination__next-arrow > a')[0]
        move_next.send_keys('\n')

        driver.implicitly_wait(1)
        time.sleep(1+random.uniform(0,1))

    except:
        pass
        
    return raw_datas

In [135]:
# 기존 요금, 현재 요금 데이터 파싱
def price_parsing(datas):
    target_prices = []
    prices = [] 
    
    try:
        for i in range(len(datas[0])):
            target_prices.append(datas[0][i][2])

        for i in range(len(target_prices)):
            re_prices = re.findall('\d+', target_prices[i])
            base_curr_prices = [] 

            if len(re_prices) == 0:
                base_curr_prices.append('')
                base_curr_prices.append('')

            elif len(re_prices) == 2:
                base_curr_prices.append(re_prices[0] + re_prices[1])
                base_curr_prices.append('')

            else:
                base_curr_prices.append(re_prices[0] + re_prices[1])
                base_curr_prices.append(re_prices[2] + re_prices[3])

            prices.append(base_curr_prices)
            
    except:
        prices.append('')
            
    return prices

In [152]:
# 현재 페이지만 스크래핑
def onePage_scraping():
    datas = get_hotelDatas()

    re_prices = price_parsing(datas)

    raw_dicts = {'hotel_name':'', 'star':'', 'base_price':'', 'cur_price':'', 'grade':'', 'good_review':'', 'bad_review':'', 'address':''}
    raw_datas = []
    hotelName_idx, star_idx, grade_idx, addr_idx, review_idx, = 0, 1, 3, 4, 5

    try:
        for i in range(len(datas[0])):

            raw_dicts['hotel_name'] = datas[0][i][hotelName_idx]
            raw_dicts['star'] = datas[0][i][star_idx]
            raw_dicts['base_price'] = re_prices[i][0]
            raw_dicts['cur_price'] = re_prices[i][1]
            raw_dicts['grade'] = datas[0][i][grade_idx]
            raw_dicts['good_review'] = datas[0][i][review_idx][0]
            raw_dicts['bad_review'] = datas[0][i][review_idx+1][0]
            raw_dicts['address'] = datas[0][i][addr_idx]

            raw_datas.append(raw_dicts.copy())

    except Exception as TypeError:
           pass
        
    finally: 
            return raw_datas
    
raw_datas = onePage_scraping()

5 페이지 완료


In [153]:
# get_fields(raw_datas)
raw_datas

[{'hotel_name': '제주 게스트하우스 파티',
  'star': '',
  'base_price': '50000',
  'cur_price': '',
  'grade': '',
  'good_review': '',
  'bad_review': '',
  'address': '서귀포, 안덕면 사계북로41번길 32'},
 {'hotel_name': '퓨어스테이',
  'star': '',
  'base_price': '',
  'cur_price': '',
  'grade': '',
  'good_review': '',
  'bad_review': '',
  'address': '제주, 제주 제주시 신대로 68'},
 {'hotel_name': '빅썸 호스텔',
  'star': '',
  'base_price': '199292',
  'cur_price': '89760',
  'grade': '',
  'good_review': '',
  'bad_review': '',
  'address': '서귀포, 제주특별자치도 서귀포시 남원읍 태위로 984-7'},
 {'hotel_name': '걸리버 여행 펜션',
  'star': '',
  'base_price': '',
  'cur_price': '',
  'grade': '',
  'good_review': '',
  'bad_review': '',
  'address': '서귀포, 안덕면 화순서서로64번길 69'},
 {'hotel_name': '타이 스토리',
  'star': '',
  'base_price': '89000',
  'cur_price': '',
  'grade': '8.5',
  'good_review': '9시에 조식을 가져다 주셔서 잘 먹었습니다. (가벼운 식사입니다)\n아늑하고 편안했어요\n나무랑 조용한 곳을 찾았는데, 생각한 그대로라 정말 만족했습니다. 특히 화장실이 나무로 이뤄어져있는데, 물에 닿으니 좋은 냄새가 나서 좋았어요. 숲은 아니지만 제주도에서 나무를 느끼고 싶다

In [154]:
def get_fields(raw_datas):

    name = [raw_datas[i]['hotel_name'] for i in range(len(raw_datas))]
    star = [raw_datas[i]['star'] for i in range(len(raw_datas))]
    basePrice = [raw_datas[i]['base_price'] for i in range(len(raw_datas))]
    currentPrice = [raw_datas[i]['cur_price'] for i in range(len(raw_datas))]
    grade = [raw_datas[i]['grade'] for i in range(len(raw_datas))]
    good_review = [raw_datas[i]['good_review'] for i in range(len(raw_datas))]
    bad_review = [raw_datas[i]['bad_review'] for i in range(len(raw_datas))]
    address = [raw_datas[i]['address'] for i in range(len(raw_datas))]
    
    data_pd = pd.DataFrame({'name': name, 'star': star, 'basePrice': basePrice, 'currentPrice': currentPrice, 'grade': grade, 'goodReview': good_review, 'badReview': bad_review, 'address': address })
    
    return data_pd

res_jeju_hotel = get_fields(raw_datas)
res_jeju_hotel

,name,star,basePrice,currentPrice,grade,goodReview,badReview,address
0,제주 게스트하우스 파티,,50000,,,,,"서귀포, 안덕면 사계북로41번길 32"
1,퓨어스테이,,,,,,,"제주, 제주 제주시 신대로 68"
2,빅썸 호스텔,,199292,89760,,,,"서귀포, 제주특별자치도 서귀포시 남원읍 태위로 984-7"
3,걸리버 여행 펜션,,,,,,,"서귀포, 안덕면 화순서서로64번길 69"
4,타이 스토리,,89000,,8.5,9시에 조식을 가져다 주셔서 잘 먹었습니다. (가벼운 식사입니다)\n아늑하고 편안했...,욕실이 외풍으로 좀 춥네요\n없습니당!\n교통이 조금 불편\n한 건물에 가벽을 설치...,"서귀포, 표선면 서성일로 118-32"
5,,,,,,매니저님이 친절했어요우~ 외쿡인도 많고 사람들 다 쏘 친절~\n이층침댄데 사다리가 ...,저희방 욕실 문 안잠겼어요ㅠㅠ 매니저님이 좀 얄밉다..?ㅎㅎ넝담~/^^\n-\n침대...,
6,Hannahstay in Jeju Guesthouse,,60000,52800,9.1,"침대마다 커튼, 전기장판이 있어서 너무 좋았구요~ 저녁 파티때 너무 재밌었어요~^^...","전혀 없어요~\n없어요\n첫 문의전화시 직원의 안내는 친절했지만,교통안내는 좀더 생...","제주, 6, Gwangyang 1-gil"
7,숨 게스트하우스 제주공항,,43800,,8.1,직원분들이 정~~~말 친절하셨어요! 6인 도미토리에서 숙박했는데 지금까지 가본 호스...,글쎄요?\n딱히 없었어요\n하나도 없다\n없어요\n숙소 침대주변 관리가 미흡합니다....,"제주, 서광로5길 2-2"
8,백패커스홈,,39600,,9.0,국내 외국 합쳐서 게스트하우스만 30군데정도 가봤는데 그중에서도 최고였습니다. 방이...,없습니다\n없습니다.\n저녁늦게 도착을했는데 일찍못가서 아쉽네요\n없어요 ㅎㅎ\n,"서귀포, 중정로 24"
9,뱅디가름 게스트하우스,,56000,,9.7,스텝들이 너무 좋았고\n시설들도 있을 것 다 있어서 편안하게 지내다 갑니당!!\n근...,개선점이라기 보다는 호텔시설을 바라실분들은 별로일수있지만 여행자의 입장으로서는 부족...,"서귀포, 성산읍 서성일로1222번길 17"


In [155]:
res_jeju_hotel.to_csv("../datas/jeju_hotel_datas.csv", encoding='utf-8-sig', header=False, mode='a') # 이어서 저장

---